<a href="https://colab.research.google.com/github/vivekvision/PySparkDataWrangling/blob/main/MultiDimentionalData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 50.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a60c94a3f14036d41324b9d1676f024890e1c42e1e47a572feac5ad4254fdf87
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [37]:
import pandas as pd
import pyspark.sql.functions as sf

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("testApp")\
        .getOrCreate()

In [38]:
df = pd.read_excel("./MultiDimensionalSample.xlsx", sheet_name="Sheet1", header=[0,1,2,3], index_col=[0,1,2,3],dtype=str)

In [39]:
df

FY2020                                    \
                              December                  November           
                            TotalCases OverRun Budget TotalCases OverRun   
                                   YTD     YTD    YTD        YTD     YTD   
Asia Singapore Sub A Item 1         34      65     45         30      60   
                     Item 2         34      65     45         30      60   
                     Item 3         34      65     45         30      60   
               Sub B Item 1         34      65     45         30      60   
                     Item 2         34      65     45         30      60   
                     Item 3         34      65     45         30      60   
EUR  France    Sub A Item 1         34      65     45         30      60   
                     Item 2         34      65     45         30      60   
                     Item 3         34      65     45         30      60   
               Sub B Item 1         34      65     45         30      60   
                     Item 2         34      65     45         30      60   
                     Item 3         34      65     45         30      60   

                                                              
                                      October                 
                            Budget TotalCases OverRun Budget  
                               YTD        YTD     YTD    YTD  
Asia Singapore Sub A Item 1     34         30      60     34  
                     Item 2     34         30      60     34  
                     Item 3     34         30      60     34  
               Sub B Item 1     34         30      60     34  
                     Item 2     34         30      60     34  
                     Item 3     34         30      60     34  
EUR  France    Sub A Item 1     34         30      60     34  
                     Item 2     34         30      60     34  
                     Item 3     34         30      60     34  
               Sub B Item 1     34         30      60     34  
                     Item 2     34         30      60     34  
                     Item 3     34         30      60     34

In [40]:
# No concept of index in Spark DataFrame, in order to preserve index columns assign index to a column with reset_index  call
df.reset_index(drop=False,inplace=True)

In [41]:
s_df = spark.createDataFrame(df.astype(str), schema=None)

In [42]:
s_df.show()

+-----------------------+-----------------------+-----------------------+-----------------------+-------------------------------------------+----------------------------------------+---------------------------------------+-------------------------------------------+----------------------------------------+---------------------------------------+------------------------------------------+---------------------------------------+--------------------------------------+
|('level_0', '', '', '')|('level_1', '', '', '')|('level_2', '', '', '')|('level_3', '', '', '')|('FY2020', 'December', 'TotalCases', 'YTD')|('FY2020', 'December', 'OverRun', 'YTD')|('FY2020', 'December', 'Budget', 'YTD')|('FY2020', 'November', 'TotalCases', 'YTD')|('FY2020', 'November', 'OverRun', 'YTD')|('FY2020', 'November', 'Budget', 'YTD')|('FY2020', 'October', 'TotalCases', 'YTD')|('FY2020', 'October', 'OverRun', 'YTD')|('FY2020', 'October', 'Budget', 'YTD')|
+-----------------------+-----------------------+-----------

In [43]:
s_df = s_df.withColumnRenamed(s_df.columns[0], "Region")
s_df = s_df.withColumnRenamed(s_df.columns[1], "Country")
s_df = s_df.withColumnRenamed(s_df.columns[2], "Sub")
s_df = s_df.withColumnRenamed(s_df.columns[3], "Item")

In [44]:
# remove all special char from column names
s_df = s_df.select([sf.col(x).alias(x.replace(" ","")) for x in s_df.columns])
s_df = s_df.select([sf.col(x).alias(x.replace("(","")) for x in s_df.columns])
s_df = s_df.select([sf.col(x).alias(x.replace(")","")) for x in s_df.columns])
s_df = s_df.select([sf.col(x).alias(x.replace("'","")) for x in s_df.columns])


In [45]:
# replace comma with underscore 
s_df = s_df.select([sf.col(x).alias(x.replace(",","_")) for x in s_df.columns])

In [46]:
# Unviot 
index_cols=["Region", "Country", "Sub", "Item"]
pivot_cols=[c for c in s_df.columns if c not in index_cols]

unPivotExp = "stack("+str(len(pivot_cols))+","+",".join(["'{"+str(i)+"}',{"+str(i)+"}" for i in range(len(pivot_cols))])+") as (Param, Value)"
s_df = s_df.selectExpr(*index_cols, unPivotExp.format(*pivot_cols))

In [47]:
# Parm column 

s_df = s_df.withColumn('Year', sf.split(sf.col('Param'),'_').getItem(0))
s_df = s_df.withColumn('Month', sf.split(sf.col('Param'),'_').getItem(1))
s_df = s_df.withColumn('Metrics', sf.split(sf.col('Param'),'_').getItem(2))
s_df = s_df.withColumn('ComulativeFreq', sf.split(sf.col('Param'),'_').getItem(3))
s_df = s_df.drop('Param')
s_df = s_df.withColumnRenamed('Value', 'Measure')

In [48]:
s_df.show()

+------+---------+-----+------+-------+------+--------+----------+--------------+
|Region|  Country|  Sub|  Item|Measure|  Year|   Month|   Metrics|ComulativeFreq|
+------+---------+-----+------+-------+------+--------+----------+--------------+
|  Asia|Singapore|Sub A|Item 1|     34|FY2020|December|TotalCases|           YTD|
|  Asia|Singapore|Sub A|Item 1|     65|FY2020|December|   OverRun|           YTD|
|  Asia|Singapore|Sub A|Item 1|     45|FY2020|December|    Budget|           YTD|
|  Asia|Singapore|Sub A|Item 1|     30|FY2020|November|TotalCases|           YTD|
|  Asia|Singapore|Sub A|Item 1|     60|FY2020|November|   OverRun|           YTD|
|  Asia|Singapore|Sub A|Item 1|     34|FY2020|November|    Budget|           YTD|
|  Asia|Singapore|Sub A|Item 1|     30|FY2020| October|TotalCases|           YTD|
|  Asia|Singapore|Sub A|Item 1|     60|FY2020| October|   OverRun|           YTD|
|  Asia|Singapore|Sub A|Item 1|     34|FY2020| October|    Budget|           YTD|
|  Asia|Singapor